<a href="https://colab.research.google.com/github/slover1126/slover1126/blob/main/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#서울 지역의 아파트 정보를 받아서 CSV 파일로 저장하는 코드

# ==========================================
# [Step 1] 라이브러리 및 설정
# ==========================================
!pip install tqdm

import requests #데이터 요청 및 수신 라이브러리
import pandas as pd
import xml.etree.ElementTree as ET #API가 주는 데이터 형식(XML)을 파이썬이 이해하도록 번역하는 라이브러리
from datetime import datetime_CAPI #날짜와 시간을 다루는 라이브러
import time #시간 딜레이를 주기 위한 라이브러리
from tqdm import tqdm #파이썬의 진행률 표시 막대 표시 라이브러리(생략 가능)
from google.colab import drive #구글 드라이브에 저장하기 위한 라이브러리

drive.mount('/content/drive') #구글 드라이브를 내 컴퓨터의 폴더처럼 연결(마운트)

# ==========================================
# [Step 2] 사용자 설정
# ==========================================

# 인증키 (Decoding 키)
service_key = "a78283ca2ca93e142226c0dd5cacda375ebba39cfa8b650d9deef4e527128906"  #인증 키(공공데이터 포털에서 인증 후 발급받기)
save_path = "/content/drive/MyDrive/seoul_apartment_trade_22_25.csv"  #결과물을 저장할 파일 이름과 경로(.csv 앞을 바꾸면 저장할 파일 이름 바꿀 수 있음)

# ==========================================
# [Step 3] 함수 정의
# ==========================================

base_url = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade" #국토교통부 아파트 실거래가 API 기본주소

seoul_gu_codes = {
    '11110': '종로구', '11140': '중구', '11170': '용산구', '11200': '성동구', '11215': '광진구',
    '11230': '동대문구', '11260': '중랑구', '11290': '성북구', '11305': '강북구', '11320': '도봉구',
    '11350': '노원구', '11380': '은평구', '11410': '서대문구', '11440': '마포구', '11470': '양천구',
    '11500': '강서구', '11530': '구로구', '11545': '금천구', '11560': '영등포구', '11590': '동작구',
    '11620': '관악구', '11650': '서초구', '11680': '강남구', '11710': '송파구', '11740': '강동구'
}   #서울시 25개 자치구의 이름과 법정동 코드(5자리)를 짝지어 놓은 딕셔너리

#날짜 리스트 생성
def get_target_months(start_year, end_year): #만약 만들려는 날짜가 현재 시간보다 미래라면 멈추기.
    months = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if datetime(year, month, 1) > datetime.now(): break
            months.append(f"{year}{month:02d}") #02d:날짜 자릿수 맞춰주는 역할
    return months

def parse_xml(response_text, gu_name):
    try:
        root = ET.fromstring(response_text)


        res_code = root.find('.//resultCode')
        if res_code is not None and res_code.text != '000':
            print(f"\n[API Error] {root.find('.//resultMsg').text}")
            return []

        items = root.findall('.//item')
        data_list = []

        for item in items:
            def get_val(tag):
                t = item.find(tag)
                return t.text.strip() if t is not None and t.text else ""

            row = {
                'Gu_Name': gu_name,
                'Dong': get_val('umdNm'),
                'Jibun': get_val('jibun'),
                'Apartment_Name': get_val('aptNm'),
                'Price_String': get_val('dealAmount'),
                'Size_m2': get_val('excluUseAr'),
                'Floor': get_val('floor'),
                'Construction_Year': get_val('buildYear'),
                'Year_Sold': get_val('dealYear'),
                'Month_Sold': get_val('dealMonth'),
                'Day_Sold': get_val('dealDay'),
            }
            data_list.append(row)
        return data_list
    except Exception as e:
        return []

# ==========================================
# [Step 4] 실행 (Main Loop)
# ==========================================

target_months = get_target_months(2022, 2025)
all_data = []

print(f"서울시 25개 구 x {len(target_months)}개월 데이터 수집 시작...")

total_iter = len(seoul_gu_codes) * len(target_months)
with tqdm(total=total_iter, desc="Progress") as pbar:
    for gu_code, gu_name in seoul_gu_codes.items():
        for month in target_months:

            params = {
                'serviceKey': service_key,
                'LAWD_CD': gu_code,
                'DEAL_YMD': month,
                'numOfRows': '9999'
            }

            try:
                resp = requests.get(base_url, params=params)
                if resp.status_code == 200:
                    rows = parse_xml(resp.text, gu_name)
                    all_data.extend(rows)
            except Exception as e:
                pass

            pbar.update(1)
            time.sleep(0.05)

# ==========================================
# [Step 5] 저장
# ==========================================

if len(all_data) > 0:
    df = pd.DataFrame(all_data)
    df['Price_Won'] = df['Price_String'].str.replace(',', '').astype(float) * 10000
    df['Address_Full'] = "서울특별시 " + df['Gu_Name'] + " " + df['Dong'] + " " + df['Jibun']
    df = df.drop_duplicates()

    print("\n" + "="*50)
    print(f"수집 성공! 총 {len(df)}건")
    print(df[['Apartment_Name', 'Price_Won', 'Address_Full']].head())

    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료: {save_path}")
else:
    print("\n데이터가 없습니다. 에러 메시지를 확인해주세요.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
서울시 25개 구 x 48개월 데이터 수집 시작...


Progress: 100%|██████████| 1200/1200 [14:49<00:00,  1.35it/s]



수집 성공! 총 178737건
  Apartment_Name     Price_Won          Address_Full
0        삼전솔하임2차  1.020000e+08  서울특별시 종로구 숭인동 296-19
1            아남1  1.180000e+09      서울특별시 종로구 명륜2가 4
2          이화에수풀  1.950000e+08  서울특별시 종로구 연건동 195-10
3      경희궁의아침3단지  2.000000e+09      서울특별시 종로구 내수동 72
4        포레스트힐시티  1.900000e+08    서울특별시 종로구 효제동 65-2
저장 완료: /content/drive/MyDrive/seoul_apartment_trade_22_25.csv


In [ ]:
# =============================================================================
# [Phase 2] 주소 기반 좌표 변환 (Geocoding)
# -----------------------------------------------------------------------------
# 1. 목적: 헤도닉 모형의 주요 설명변수인 '거리 변수(지하철, 공원, 수변 등)' 산출을 위한 기초 데이터 확보
# 2. 방법: Kakao Map REST API를 활용하여 텍스트 형태의 주소를 위도(Latitude)·경도(Longitude)로 변환
# 3. 효율화: 18만 건 전체를 호출하는 대신, '고유 주소(Unique Address)'만 추출하여 API 호출 횟수 및 소요 시간 최소화
# =============================================================================


# ==========================================
# [Step 1] 라이브러리 및 설정
# ==========================================
import requests
import pandas as pd
from tqdm import tqdm
from google.colab import drive
import time

# 드라이브 마운트 (이미 되어있으면 패스됩니다)
drive.mount('/content/drive')

#  REST API 키
kakao_api_key = "6a0a344423c94f3c531c8a6ee2fc69ea"

# 파일 경로 (Phase 1에서 저장한 파일)
input_path = "/content/drive/MyDrive/seoul_apartment_trade_22_25.csv"
output_path = "/content/drive/MyDrive/seoul_apartment_phase2_geocoded.csv"

# ==========================================
# [Step 2] 데이터 불러오기 및 준비
# ==========================================
print("데이터를 불러오는 중입니다...")
df = pd.read_csv(input_path)

# 중복 주소 제거하고 '고유 주소'만 추출 (API 호출 횟수 절약!)
unique_addresses = df['Address_Full'].unique()
print(f"전체 데이터: {len(df)}건")
print(f"변환할 고유 주소: {len(unique_addresses)}건")
print("-" * 50)

# ==========================================
# [Step 3] 좌표 변환 함수 정의 (카카오 API)
# ==========================================
def get_lat_lon(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}
    params = {"query": address}

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            # 검색 결과가 있으면 첫 번째 결과의 좌표 반환
            if result['documents']:
                x = result['documents'][0]['x'] # 경도 (Longitude)
                y = result['documents'][0]['y'] # 위도 (Latitude)
                return y, x
            else:
                return None, None # 검색 결과 없음
        else:
            return None, None # API 에러
    except:
        return None, None # 네트워크 에러

# ==========================================
# [Step 4] 좌표 변환 실행 (Unique Address 기준)
# ==========================================
coord_dict = {} # 주소: (위도, 경도) 를 저장할 사전

print("좌표 변환을 시작합니다... (시간이 좀 걸립니다)")

# tqdm으로 진행률 보기
for addr in tqdm(unique_addresses):
    # 빈 주소면 패스
    if pd.isna(addr) or addr.strip() == "":
        continue

    lat, lon = get_lat_lon(addr)
    coord_dict[addr] = {'Latitude': lat, 'Longitude': lon}


# ==========================================
# [Step 5] 결과 합치기 및 저장
# ==========================================
print("\n좌표 데이터를 원본 데이터와 병합 중...")

# 1. 좌표 사전을 데이터프레임으로 변환
coord_df = pd.DataFrame.from_dict(coord_dict, orient='index')
coord_df.index.name = 'Address_Full'
coord_df.reset_index(inplace=True)

# 2. 원본 데이터(df)에 좌표 데이터(coord_df)를 주소 기준으로 붙이기 (Left Join)
final_df = pd.merge(df, coord_df, on='Address_Full', how='left')

# 3. 결과 확인
print(f"변환 완료! 총 {len(final_df)}건")
print(final_df[['Address_Full', 'Latitude', 'Longitude']].head())

# 4. 결측치(좌표 못 구한 곳) 확인
missing = final_df['Latitude'].isna().sum()
print(f"\n좌표를 못 구한 데이터: {missing}건")

# 5. 저장
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"저장 완료: {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터를 불러오는 중입니다...
전체 데이터: 178737건
변환할 고유 주소: 7252건
--------------------------------------------------
좌표 변환을 시작합니다... (시간이 좀 걸립니다)


100%|██████████| 7252/7252 [24:41<00:00,  4.90it/s]



좌표 데이터를 원본 데이터와 병합 중...
변환 완료! 총 178737건
           Address_Full          Latitude         Longitude
0  서울특별시 종로구 숭인동 296-19  37.5724071549574  127.016729343206
1      서울특별시 종로구 명륜2가 4  37.5855421290535  126.999413978947
2  서울특별시 종로구 연건동 195-10  37.5754399031459  127.001845558431
3      서울특별시 종로구 내수동 72   37.573660668544  126.972093823848
4    서울특별시 종로구 효제동 65-2  37.5742170733379  127.001834547682

좌표를 못 구한 데이터: 74건
저장 완료: /content/drive/MyDrive/seoul_apartment_phase2_geocoded.csv


In [ ]:
# =============================================================================
# [Phase 3] 헤도닉 모형 환경 변수(Environmental Variables) 산출
# -----------------------------------------------------------------------------
# 1. 목적: 아파트 가격 결정 요인 중 '입지 및 편의시설 접근성'을 정량화하여 분석 데이터 구축
# 2. 산출 변수:
#    - Dist_Subway: 가장 가까운 지하철역까지의 최단 거리 (m)
#    - Dist_Green: 가장 가까운 공원/녹지까지의 최단 거리 (m)
#    - Dist_CBD: 도심(시청)까지의 직선 거리 (m)
# 3. 방법론:
#    - Kakao Local API를 활용한 관심 지점(POI) 좌표 전수 수집
#    - cKDTree(Nearest Neighbor) 알고리즘을 적용하여 18만 건의 대용량 거리 연산 최적화
# =============================================================================
# =============================================================================
# [Phase 3] (수정판) 주요 시설물 거리 계산기 - 공원 검색 버그 수정
# -----------------------------------------------------------------------------
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.spatial import cKDTree
from google.colab import drive
import time

# 드라이브 마운트
drive.mount('/content/drive')

# ★ 태삣삐님의 REST API 키
kakao_api_key = "6a0a344423c94f3c531c8a6ee2fc69ea"

# 파일 경로
input_path = "/content/drive/MyDrive/seoul_apartment_phase2_geocoded.csv"
output_path = "/content/drive/MyDrive/seoul_apartment_phase3_dist.csv"

# 1. 데이터 로드
print("Phase 2 데이터를 불러옵니다...")
df = pd.read_csv(input_path)
df_clean = df.dropna(subset=['Latitude', 'Longitude']).copy()

sample_addr = df_clean['Address_Full'].iloc[0].split()
city_name = sample_addr[0]
gu_list = df_clean['Gu_Name'].unique()

print(f"🏙️ 대상 도시: {city_name}")

# 2. 시설물 수집 함수 (여기가 수정되었습니다!)
def collect_poi(keyword, code, gu_names):
    places = []
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    print(f"\n🔍 '{keyword}' 위치 정보를 수집 중...")
    for gu in tqdm(gu_names):
        query = f"{city_name} {gu} {keyword}"

        for page in range(1, 4):
            # ★ [수정] 코드가 있을 때만 params에 넣도록 변경
            params = {"query": query, "page": page, "size": 15}
            if code:
                params["category_group_code"] = code

            try:
                resp = requests.get(url, headers=headers, params=params)
                if resp.status_code == 200:
                    data = resp.json()
                    for doc in data['documents']:
                        places.append({'Name': doc['place_name'], 'Lat': float(doc['y']), 'Lon': float(doc['x'])})
                    if data['meta']['is_end']: break
                else:
                    time.sleep(0.1)
            except:
                pass
            time.sleep(0.05)

    result_df = pd.DataFrame(places).drop_duplicates(subset=['Name'])
    print(f"   -> 총 {len(result_df)}개의 {keyword} 발견!")
    return result_df

# 3. 수집 실행
subway_df = collect_poi("지하철역", "SW8", gu_list)
park_df = collect_poi("공원", "", gu_list) # 이제 정상 작동합니다!

# 4. CBD 위치 (시청)
cbd_df = pd.DataFrame()
try:
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    resp = requests.get(url, headers={"Authorization": f"KakaoAK {kakao_api_key}"}, params={"query": f"{city_name}청"})
    if resp.status_code == 200 and resp.json()['documents']:
        doc = resp.json()['documents'][0]
        cbd_df = pd.DataFrame([{'Name': doc['place_name'], 'Lat': float(doc['y']), 'Lon': float(doc['x'])}])
except:
    pass

# 5. 거리 계산 (KDTree + Haversine)
def get_nearest_distance(apt_df, poi_df, col_name):
    if poi_df.empty:
        print(f"⚠️ {col_name} 데이터가 없습니다.")
        apt_df[col_name] = np.nan
        return apt_df

    apt_coords = apt_df[['Latitude', 'Longitude']].values
    poi_coords = poi_df[['Lat', 'Lon']].values
    tree = cKDTree(poi_coords)

    dists, indexes = tree.query(apt_coords, k=1)
    nearest_poi_coords = poi_coords[indexes]

    R = 6371000
    phi1 = np.radians(apt_coords[:, 0])
    phi2 = np.radians(nearest_poi_coords[:, 0])
    dphi = np.radians(nearest_poi_coords[:, 0] - apt_coords[:, 0])
    dlambda = np.radians(nearest_poi_coords[:, 1] - apt_coords[:, 1])

    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2) * np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    apt_df[col_name] = R * c
    return apt_df

print("\n📏 거리 재계산 중...")
df_clean = get_nearest_distance(df_clean, subway_df, 'Dist_Subway')
df_clean = get_nearest_distance(df_clean, park_df, 'Dist_Green')

if not cbd_df.empty:
    cbd_lat, cbd_lon = cbd_df.iloc[0]['Lat'], cbd_df.iloc[0]['Lon']
    R = 6371000
    phi1 = np.radians(df_clean['Latitude'])
    phi2 = np.radians(cbd_lat)
    dphi = np.radians(cbd_lat - df_clean['Latitude'])
    dlambda = np.radians(cbd_lon - df_clean['Longitude'])
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2) * np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    df_clean['Dist_CBD'] = R * c
else:
    df_clean['Dist_CBD'] = np.nan

# 6. 저장
print("="*50)
print(f"🎉 수정 완료! 총 {len(df_clean)}건")
print(df_clean[['Dist_Subway', 'Dist_Green', 'Dist_CBD']].head()) # 여기 숫자가 나와야 성공!
df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"💾 저장 완료: {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Phase 2 데이터를 불러옵니다...
🏙️ 대상 도시: 서울특별시

🔍 '지하철역' 위치 정보를 수집 중...


100%|██████████| 25/25 [00:17<00:00,  1.47it/s]


   -> 총 430개의 지하철역 발견!

🔍 '공원' 위치 정보를 수집 중...


100%|██████████| 25/25 [00:18<00:00,  1.38it/s]


   -> 총 1068개의 공원 발견!

📏 거리 재계산 중...
🎉 수정 완료! 총 178663건
   Dist_Subway  Dist_Green     Dist_CBD
0    95.591505  154.494989  3412.843171
1   448.385866  314.951666  2771.216080
2   497.089216  104.813207  2257.456188
3   277.008760  121.979697   954.977018
4   361.323089  196.590451  2202.310139
💾 저장 완료: /content/drive/MyDrive/seoul_apartment_phase3_dist.csv


In [ ]:
# =================================================================================================
# [Phase 4] 입지·환경 변수 통합 구축 (수변/학교/버스)
# =================================================================================================
# 1. 기능:
#    - Kakao API: 수변공원 거리(Dist_Water), 반경 5km의 고등학교 개수(High_School_Count)
#    - 로컬 파일: 버스정류장 개수(Bus_Stop) - 중복 파일 자동 처리
# =================================================================================================

import pandas as pd
import numpy as np
import requests
import time
import glob
import os
from scipy.spatial import cKDTree
from tqdm.auto import tqdm  # 진행바(Progress Bar) 라이브러리
from google.colab import drive

# 1. 드라이브 마운트
drive.mount('/content/drive')
kakao_api_key = "6a0a344423c94f3c531c8a6ee2fc69ea"  # API 키

# =============================================================================
# [Step 1] Phase 3 데이터 로드
# =============================================================================
input_path = "/content/drive/MyDrive/seoul_apartment_phase3_dist.csv"
output_path = "/content/drive/MyDrive/seoul_apartment_phase4.csv"

print("📂 Phase 3 데이터를 불러옵니다...")
if not os.path.exists(input_path):
    raise FileNotFoundError(f"❌ Phase 3 파일이 없습니다: {input_path}")

df = pd.read_csv(input_path)
print(f"✅ 데이터 로드 성공: 총 {len(df)}건")

# 구 목록 추출 (API 검색용)
city_name = "서울특별시"
if 'Address_Full' in df.columns:
    city_name = df['Address_Full'].iloc[0].split()[0]
gu_list = df['Gu_Name'].unique()

# =============================================================================
# [Step 2] Kakao API 기반 데이터 수집 (수변, 학교) - 진행바 적용
# =============================================================================
def collect_poi_api(keywords, code, gu_names, desc):
    places = []
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    print(f"\n🔍 [{desc}] 정보 수집 시작 (API)...")

    # ★ 진행바(tqdm) 적용: 구별로 검색 진행 상황을 보여줌
    for gu in tqdm(gu_names, desc=f"{desc} 검색 중", unit="구"):
        for keyword in keywords:
            query = f"{city_name} {gu} {keyword}"
            for page in range(1, 4): # 최대 3페이지(45개)까지 검색
                try:
                    params = {"query": query, "page": page, "size": 15}
                    if code: params["category_group_code"] = code

                    resp = requests.get(url, headers=headers, params=params, timeout=5)

                    if resp.status_code == 200:
                        data = resp.json()
                        for doc in data['documents']:
                            places.append({
                                'Name': doc['place_name'],
                                'Lat': float(doc['y']),
                                'Lon': float(doc['x'])
                            })
                        if data['meta']['is_end']: break
                except Exception as e:
                    pass
                time.sleep(0.05) # API 과부하 방지

    return pd.DataFrame(places).drop_duplicates(subset=['Name', 'Lat', 'Lon'])

# 1. 수변 데이터 (한강, 천, 호수)
water_df = collect_poi_api(["한강공원", "수변공원", "천", "호수공원"], "", gu_list, "수변 시설")

# 2. 고등학교 데이터
hs_df = collect_poi_api(["고등학교"], "SC4", gu_list, "고등학교")

# 3. 거리/개수 계산 함수 (Vectorized)
def calc_features(apt_df, poi_df, mode='dist', radius_km=5, col_name='Target'):
    if poi_df.empty:
        print(f"⚠️ {col_name}: 매칭된 시설이 없습니다.")
        apt_df[col_name] = np.nan
        return apt_df

    apt_coords = np.radians(apt_df[['Latitude', 'Longitude']].values)
    poi_coords = np.radians(poi_df[['Lat', 'Lon']].values)
    tree = cKDTree(poi_coords)

    if mode == 'dist': # 최단 거리 (m)
        dists, _ = tree.query(apt_coords, k=1)
        apt_df[col_name] = dists * 6371000
    elif mode == 'count': # 반경 내 개수
        indexes = tree.query_ball_point(apt_coords, r=radius_km/6371)
        apt_df[col_name] = [len(i) for i in indexes]
    return apt_df

# 계산 실행
print("\n📐 [연산] 수변 거리 및 고등학교 개수 계산 중...")
df = calc_features(df, water_df, mode='dist', col_name='Dist_Water')
df = calc_features(df, hs_df, mode='count', radius_km=5, col_name='High_School_Count')

# =============================================================================
# [Step 3] 버스정류장 파일 로드 & 중복 해결
# =============================================================================
print("\n🚌 [핵심] 버스정류장 파일 로드 중...")

search_pattern = "/content/drive/MyDrive/seoul_bus(20260108)*"
found_files = glob.glob(search_pattern)

if not found_files:
    raise FileNotFoundError("❌ 'seoul_bus' 관련 파일을 찾을 수 없습니다.")

# 최신 파일 자동 선택
bus_file_path = max(found_files, key=os.path.getmtime)
print(f"✅ 선택된 파일(최신): {os.path.basename(bus_file_path)}")

try:
    if bus_file_path.endswith('.xlsx'):
        bus_df = pd.read_excel(bus_file_path)
    else:
        try: bus_df = pd.read_csv(bus_file_path, encoding='cp949')
        except: bus_df = pd.read_csv(bus_file_path, encoding='utf-8')

    print(f"   -> 정류장 데이터 {len(bus_df)}개 로드 완료")

    # 좌표 컬럼 찾기
    cols = bus_df.columns
    x_col = next((c for c in cols if 'X' in str(c) or '경도' in str(c)), 'X좌표')
    y_col = next((c for c in cols if 'Y' in str(c) or '위도' in str(c)), 'Y좌표')
    print(f"   -> 좌표 매핑: X='{x_col}', Y='{y_col}'")

except Exception as e:
    raise ValueError(f"파일 열기 실패: {e}")

# =============================================================================
# [Step 4] 버스정류장 반경 400m 개수 산출
# =============================================================================
print("🚀 반경 400m 내 버스정류장 개수 카운팅...")

apt_coords = np.radians(df[['Latitude', 'Longitude']].values)
bus_coords = np.column_stack((bus_df[y_col].values, bus_df[x_col].values)) # 위도, 경도
bus_coords_rad = np.radians(bus_coords)

tree_bus = cKDTree(bus_coords_rad)
radius_rad = 0.4 / 6371 # 400m
indexes = tree_bus.query_ball_point(apt_coords, r=radius_rad)

df['Bus_Stop'] = [len(i) for i in indexes]

# =============================================================================
# [Final] 결과 저장
# =============================================================================
print("="*60)
print(f"🎉 Phase 4 최종 완료! 총 {len(df)}건")
print("🔍 결과 미리보기:")
print(df[['Address_Full', 'Dist_Water', 'High_School_Count', 'Bus_Stop']].head())

df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n💾 파일 저장 완료: {output_path}")

Mounted at /content/drive
📂 Phase 3 데이터를 불러옵니다...
✅ 데이터 로드 성공: 총 178663건

🔍 [수변 시설] 정보 수집 시작 (API)...


수변 시설 검색 중:   0%|          | 0/25 [00:00<?, ?구/s]


🔍 [고등학교] 정보 수집 시작 (API)...


고등학교 검색 중:   0%|          | 0/25 [00:00<?, ?구/s]


📐 [연산] 수변 거리 및 고등학교 개수 계산 중...

🚌 [핵심] 버스정류장 파일 로드 중...
✅ 선택된 파일(최신): seoul_bus(20260108).xlsx
   -> 정류장 데이터 11237개 로드 완료
   -> 좌표 매핑: X='X좌표', Y='Y좌표'
🚀 반경 400m 내 버스정류장 개수 카운팅...
🎉 Phase 4 최종 완료! 총 178663건
🔍 결과 미리보기:
           Address_Full   Dist_Water  High_School_Count  Bus_Stop
0  서울특별시 종로구 숭인동 296-19   731.714867                 52        12
1      서울특별시 종로구 명륜2가 4  2377.660594                 52        19
2  서울특별시 종로구 연건동 195-10  1555.559376                 52        16
3      서울특별시 종로구 내수동 72  1823.958603                 46         7
4    서울특별시 종로구 효제동 65-2  1478.430464                 52        13

💾 파일 저장 완료: /content/drive/MyDrive/seoul_apartment_phase4.csv
